# Step 1: Import Libraries 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# didn't add LabEncoded since dataset used is already numerical

# Step 2: Link/Upload Dataset



In [ ]:
df = pd.read_csv("/kaggle/input/wildfire/final_dataset.csv")

print("Shape:", df.shape)
display(df.head())
print(df.columns)

# Step 3: Fix the Data

In [ ]:
df.tail()

In [ ]:
df.head()

In [ ]:
df.describe()

# Step 4: Define X (features) and y (target)
Sets the prediction target (occured) and builds the input features by dropping occured and frp

In [ ]:
y = df["occured"].astype(int)

# Drop target + leakage column
X = df.drop(columns=["occured", "frp"])

print("X shape:", X.shape)
print("y distribution:\n", y.value_counts())

# Fill missing numeric values using the median so KNN can run without errors
X = X.fillna(X.median(numeric_only=True))

# Step 5: Spitting the Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)


# Step 6: Train the KNN model (with scaling)
Scales the data (important for KNN) and trains the classifier.
Note: scaling is important if dealing with big and small numbers.

In [ ]:
knn_model = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=15))
])

knn_model.fit(X_train, y_train)


# Step 7: Test (predict on the test set)
Uses the trained model to predict labels for unseen test data.

In [ ]:
y_pred = knn_model.predict(X_test)


# Step 8: Accuracy 

In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Fire", "Fire"])
disp.plot()
plt.show()


# Step 9: Tune K and compare improvement
Tests multiple k values, finds the best one, plots accuracy vs k, then retrains using the best k and prints the new accuracy.

In [ ]:
k_values = [1,3,5,7,9,11,15,21,31,41,51]
accs = []

for k in k_values:
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=k))
    ])
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    accs.append(accuracy_score(y_test, pred))

best_k = k_values[int(np.argmax(accs))]
best_acc = max(accs)

print("K values tested:", k_values)
print("Accuracies:", accs)
print("Best K:", best_k)
print("Best Test Accuracy:", best_acc)

plt.figure()
plt.plot(k_values, accs, marker="o")
plt.xlabel("k (number of neighbors)")
plt.ylabel("Test Accuracy")
plt.title("KNN Accuracy vs k")
plt.show()

# Retrain final tuned KNN and store predictions for later evaluation (confusion matrix etc.)
knn_model_tuned = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=best_k))
])
knn_model_tuned.fit(X_train, y_train)

y_pred_tuned = knn_model_tuned.predict(X_test)

print("Tuned KNN Test Accuracy:", accuracy_score(y_test, y_pred_tuned))


# Step 10: Train a Random Forest Model (to see if we get better accuracy)
Trains a Random Forest classifier (many decision trees voting together) to capture non-linear patterns in the weather data. It often performs better than KNN on tabular datasets.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Step 9: Random Forest (Optional Improvement Step)
# Trains many decision trees and combines them to improve prediction accuracy

rf_model = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print("Random Forest Test Accuracy:", accuracy_score(y_test, y_pred_rf))

# Confusion Matrix Visualization
cm_rf = confusion_matrix(y_test, y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=["No Fire", "Fire"])
disp.plot()
plt.title("Random Forest Confusion Matrix")
plt.show()


# Step 11: Using HistGradientBoosting (Best Tabular Model)
Trying this model to see if accuracy improves, which allows us to compare which model is best suited to predict wildfire.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Step 11: Tune HistGradientBoosting (Optional)
# Tries different settings to see if boosting can beat Random Forest

param_grid = {
    "learning_rate": [0.03, 0.05, 0.1],
    "max_depth": [3, 5, 7, None],
    "max_iter": [200, 400, 600]
}

hgb = HistGradientBoostingClassifier(random_state=42)
grid = GridSearchCV(hgb, param_grid, scoring="accuracy", cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
best_hgb = grid.best_estimator_

y_pred_best_hgb = best_hgb.predict(X_test)
print("Tuned HGB Test Accuracy:", accuracy_score(y_test, y_pred_best_hgb))

cm = confusion_matrix(y_test, y_pred_best_hgb)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Fire", "Fire"])
disp.plot()
plt.title("Tuned HistGradientBoosting Confusion Matrix")
plt.show()


# Step 12: Compare all 3 models
This showcases which model was the most accurate.

In [ ]:
import matplotlib.pyplot as plt

results = {
    "KNN (tuned k)": 0.6497139491839139,
    "Random Forest": 0.6678865892646811,
    "HistGradientBoosting (tuned)": 0.6559818273599193
}

models = list(results.keys())
scores = list(results.values())

best_model = max(results, key=results.get)
best_score = results[best_model]

plt.figure(figsize=(8,4))
bars = plt.bar(models, scores)

plt.ylim(0.5, 0.75)
plt.ylabel("Test Accuracy")
plt.title("Model Comparison (Test Accuracy)")

# Add value labels above bars
for i, v in enumerate(scores):
    plt.text(i, v + 0.003, f"{v:.3f}", ha="center")

plt.xticks(rotation=15, ha="right")
plt.show()

print(f"Best model: {best_model} with accuracy = {best_score:.4f}")


# Conclusion
Model comparison: We trained three classifiers (KNN, Random Forest, and HistGradientBoosting) on the same 80/20 split and compared their test accuracy. Random Forest achieved the best performance (0.668), followed by HistGradientBoosting (0.656) and KNN (0.650). The overall accuracy is moderate because wildfire occurrence cannot be explained by weather variables alone—fires are also influenced by ignition sources (human activity/lightning), vegetation and fuel conditions, and local geography—so the classes overlap and the model cannot perfectly separate “fire” vs “no fire” using the available features.

# Adjustable Weather Conditions to Check Fire Likelihood
This allows us to check how likely it is for a fire to happen based on weather conditions.

In [ ]:
# GUI (Day/Night theme + clean radio colors + rounded slider steps + % output)

import numpy as np
import pandas as pd
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load dataset
try:
    df
except NameError:
    df = pd.read_csv("/kaggle/input/wildfire/final_dataset.csv") 

# Prepare X/y
if "occured" not in df.columns:
    raise ValueError("Couldn't find 'occured' column. Check df.columns.")

drop_cols = ["occured"] + (["frp"] if "frp" in df.columns else [])
y = df["occured"].astype(int)
X = df.drop(columns=drop_cols)
X = X.fillna(X.median(numeric_only=True))

# Split
try:
    X_train, X_test, y_train, y_test
except NameError:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

# Train model
try:
    rf_model
except NameError:
    rf_model = RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)

feature_cols = X_train.columns.tolist()
has_daynight = "daynight_N" in feature_cols

# Weather-only sliders (exclude lat/lon + exclude wind mean)
exclude_cols = {"lat", "lon"}
weather_keywords = [
    "temp", "humidity", "rh", "wind", "precip", "rain",
    "pressure", "dewpoint", "cloud", "fwi", "fire_weather_index"
]

weather_cols = [
    c for c in feature_cols
    if c.lower() not in exclude_cols
    and c != "daynight_N"
    and any(k in c.lower() for k in weather_keywords)
]

preferred_order = [
    "temp_mean", "temp_max", "temp_min",
    "humidity_min", "humidity_mean", "humidity_max",
    "wind_speed_max",            # keep max wind
    # "wind_speed_mean",         # intentionally removed
    "precip_mean",
    "pressure_mean",
    "dewpoint_mean",
    "cloud_cover_mean",
    "fire_weather_index"
]

ordered_weather = [c for c in preferred_order if c in weather_cols]
ordered_weather += [c for c in weather_cols if c not in ordered_weather]

# Explicitly remove wind mean if it exists
ordered_weather = [c for c in ordered_weather if c != "wind_speed_mean"]

slider_features = ordered_weather[:8]  # scroll is OK

if len(slider_features) == 0:
    raise ValueError("No weather-like columns found for sliders. Check df.columns.")

friendly_label = {
    "temp_mean": "Average Temperature (°C)",
    "temp_max": "Maximum Temperature (°C)",
    "temp_min": "Minimum Temperature (°C)",

    "humidity_mean": "Average Humidity (%)",
    "humidity_min": "Minimum Humidity (%)",
    "humidity_max": "Maximum Humidity (%)",

    "wind_speed_max": "Maximum Wind Speed (m/s)",

    "precip_mean": "Rainfall / Precipitation (mm)",
    "pressure_mean": "Air Pressure (hPa)",
    "dewpoint_mean": "Dew Point (°C)",
    "cloud_cover_mean": "Cloud Cover (%)",

    "fire_weather_index": "Fire Weather Index (higher = riskier)"
}

slider_info = {
    "temp_mean": "Cold ⟵  ⟶ Hot",
    "temp_max": "Lower max ⟵  ⟶ Higher max",
    "temp_min": "Colder nights ⟵  ⟶ Warmer nights",

    "humidity_mean": "Drier ⟵  ⟶ More humid",
    "humidity_min": "Very dry ⟵  ⟶ Less dry",
    "humidity_max": "Less humid ⟵  ⟶ More humid",

    "wind_speed_max": "Lower gusts ⟵  ⟶ Stronger gusts",

    "precip_mean": "Drier ⟵  ⟶ Wetter",
    "pressure_mean": "Lower ⟵  ⟶ Higher",
    "dewpoint_mean": "Drier air ⟵  ⟶ Moister air",
    "cloud_cover_mean": "Clearer ⟵  ⟶ Cloudier",

    "fire_weather_index": "Lower danger ⟵  ⟶ Higher danger"
}

# Slider ranges/defaults (rounded) + steps (integers or 1 decimal max)
ranges, defaults, steps = {}, {}, {}
for col in slider_features:
    lo = float(X_train[col].quantile(0.01))
    hi = float(X_train[col].quantile(0.99))
    mid = float(X_train[col].median())

    lo = round(lo, 1)
    hi = round(hi, 1)
    mid = round(mid, 1)

    span = hi - lo
    step = 1.0 if span >= 20 else 0.1  # whole numbers for wide ranges, else 1 decimal

    ranges[col] = (lo, hi)
    defaults[col] = mid
    steps[col] = step

def make_theme_css(mode: str) -> str:
    if mode == "Night":
        accent = "#2f80ff"
        accent2 = "#00c2ff"
        bg1 = "#07101b"
        bg2 = "#05060a"
        panel = "rgba(10, 18, 32, 0.88)"
        border = "rgba(47, 128, 255, 0.28)"
        chip_bg = "linear-gradient(90deg, #2f80ff 0%, #00c2ff 100%)"
    else:
        accent = "#ff6a00"
        accent2 = "#ff3d00"
        bg1 = "#2a160f"
        bg2 = "#0b0b0d"
        panel = "rgba(18, 22, 30, 0.88)"
        border = "rgba(255, 106, 0, 0.24)"
        chip_bg = "linear-gradient(90deg, #ff6a00 0%, #ff3d00 100%)"

    return f"""
    <style>
      html, body {{
        background: radial-gradient(circle at top, {bg1} 0%, {bg2} 65%, #000 100%) !important;
      }}

      .gradio-container {{
        background: radial-gradient(circle at top, {bg1} 0%, {bg2} 65%, #000 100%) !important;
        color: #f3f4f6 !important;
      }}

      .block, .gr-box, .gr-panel, .gr-group, .gr-form, .wrap {{
        background: {panel} !important;
        border: 1px solid {border} !important;
        box-shadow: 0 10px 30px rgba(0,0,0,0.28) !important;
      }}

      /* Label chips (section headers like "Time of Day", slider labels, etc.) */
      label > span, .label, .gr-label, .label-wrap span, .wrap .label span {{
        background: {chip_bg} !important;
        color: #fff !important;
        border-radius: 10px !important;
        padding: 4px 10px !important;
        font-weight: 800 !important;
      }}

      /* Predict button (force override) */
      button.primary, .gr-button-primary, button[variant="primary"] {{
        background: {chip_bg} !important;
        border: none !important;
        color: #fff !important;
        font-weight: 800 !important;
      }}

      /* Sliders */
      input[type="range"]::-webkit-slider-thumb {{
        background: {accent} !important;
      }}
      input[type="range"]::-webkit-slider-runnable-track {{
        background: linear-gradient(90deg, {accent} 0%, rgba(255,255,255,0.22) 65%) !important;
      }}

      /* Radio buttons: consistent day/night styling WITHOUT messing other components */
      .gr-radio label {{
        display: inline-flex !important;
        align-items: center !important;
        gap: 10px !important;
        border: 1px solid {border} !important;
        border-radius: 12px !important;
        padding: 8px 12px !important;
        margin-right: 10px !important;
        background: rgba(255,255,255,0.04) !important;
        cursor: pointer !important;
      }}
      .gr-radio label span {{
        color: #e5e7eb !important;
        font-weight: 800 !important;
      }}
      .gr-radio input[type="radio"] {{
        accent-color: {accent} !important;
      }}
      /* Checked state (works on older DOMs too) */
      .gr-radio label input[type="radio"]:checked + span,
      .gr-radio label input[type="radio"]:checked ~ span {{
        color: #fff !important;
      }}
      .gr-radio label:has(input[type="radio"]:checked) {{
        background: {chip_bg} !important;
        border: none !important;
      }}
    </style>
    """

def predict_fire(day_or_night, *values):
    user_vals = dict(zip(slider_features, values))

    row_dict = {c: float(X_train[c].median()) for c in feature_cols}

    if has_daynight:
        row_dict["daynight_N"] = 1.0 if day_or_night == "Day" else 0.0

    for k, v in user_vals.items():
        row_dict[k] = float(v)

    row = pd.DataFrame([row_dict], columns=feature_cols)
    proba_fire = float(rf_model.predict_proba(row)[0, 1])

    label = "🔥 Fire likely" if proba_fire >= 0.5 else "✅ No fire likely"
    percent = f"{proba_fire * 100:.1f}%"
    return label, percent

base_theme = gr.themes.Soft(primary_hue="orange", secondary_hue="red", neutral_hue="slate")

with gr.Blocks(theme=base_theme) as demo:
    theme_html = gr.HTML(value=make_theme_css("Day"))

    gr.Markdown("# 🔥 Wildfire Likelihood Simulator")
    gr.Markdown("Adjust weather conditions and choose **Day/Night**. Click **Predict** for probability.")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Controls")

            daynight = gr.Radio(
                choices=["Day", "Night"],
                value="Day",
                label="Time of Day"
            )

            slider_inputs = []
            for col in slider_features:
                lo, hi = ranges[col]
                slider_inputs.append(
                    gr.Slider(
                        minimum=lo,
                        maximum=hi,
                        value=defaults[col],
                        step=steps[col],
                        label=friendly_label.get(col, col),
                        info=slider_info.get(col, "Low ⟵  ⟶ High")
                    )
                )

        with gr.Column(scale=1):
            gr.Markdown("### Output")
            out_label = gr.Textbox(label="Prediction", interactive=False)
            out_prob = gr.Textbox(label="Probability of Fire (%)", interactive=False)
            predict_btn = gr.Button("Predict", variant="primary")

    daynight.change(fn=make_theme_css, inputs=daynight, outputs=theme_html)
    predict_btn.click(fn=predict_fire, inputs=[daynight] + slider_inputs, outputs=[out_label, out_prob])

demo.launch(debug=True)
